# --- Day 3: Gear Ratios ---
You and the Elf eventually reach a gondola lift station; he says the gondola lift will take you up to the water source, but this is as far as he can bring you. You go inside.

It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.

"Aaah!"

You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)

Here is an example engine schematic:
```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..

```
In this schematic, two numbers are not part numbers because they are not adjacent to a symbol: 114 (top right) and 58 (middle right). Every other number is adjacent to a symbol and so is a part number; their sum is 4361.

Of course, the actual engine schematic is much larger. What is the sum of all of the part numbers in the engine schematic?


## --- Part Two ---
The engineer finds the missing part and installs it in the engine! As the engine springs to life, you jump in the closest gondola, finally ready to ascend to the water source.

You don't seem to be going very fast, though. Maybe something is still wrong? Fortunately, the gondola has a phone labeled "help", so you pick it up and the engineer answers.

Before you can explain the situation, she suggests that you look out the window. There stands the engineer, holding a phone in one hand and waving with the other. You're going so slowly that you haven't even left the station. You exit the gondola.

The missing part wasn't the only issue - one of the gears in the engine is wrong. A gear is any * symbol that is adjacent to exactly two part numbers. Its gear ratio is the result of multiplying those two numbers together.

This time, you need to find the gear ratio of every gear and add them all up so that the engineer can figure out which gear needs to be replaced.

Consider the same engine schematic again:
```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..

```
In this schematic, there are two gears. The first is in the top left; it has part numbers 467 and 35, so its gear ratio is 16345. The second gear is in the lower right; its gear ratio is 451490. (The * adjacent to 617 is not a gear because it is only adjacent to one part number.) Adding up all of the gear ratios produces 467835.

What is the sum of all of the gear ratios in your engine schematic?

In [1]:
from pathlib import Path
import os

yr = 2023
d = 3

inp_path = os.path.join(Path(os.path.abspath("")).parents[1], 
             'Input', '{}'.format(yr), 
             '{}.txt'.format(d))


with open(inp_path, 'r') as file:
    inp = file.read()

In [2]:
def is_number(c):
  numbers = ['0','1','2','3','4','5','6','7','8','9']
  return c in numbers

def is_symbol(c):
  return (c is not None and isinstance(c,str)
  and len(c)==1
  and not is_number(c)
  and c != '.')


def format_input(inp):
  '''Turn text input into a dictionary'''

  d = {}
  for i, l in enumerate(inp.split('\n')):
    for j, c in enumerate(l):
      if c != '.':
          d[(i,j)] = c
  return d


In [3]:
def get_part_numbers(formatted_inp):

  max_i =  max([x[0] for x in formatted_inp.keys()])
  max_j = max([x[1] for x in formatted_inp.keys()])

  def check_adjacent_symbols(i, j):
    for ii in [i-1,i,i+1]:
      for jj in [j-1, j, j+1]:
        if not (ii == i and jj == j):
          if is_symbol(formatted_inp.get((ii,jj), None)):
            return True
    return False

  found_part_numbers = []
  found_part_number_indices = []
  for i in range(max_i+1):
    for j in range(max_j+1):
      if is_number(formatted_inp.get((i,j), None)):
        k = j
        cur_number_chars = []
        cur_number_char_indices = []
        adjacent_symbol = False
        while (is_number(formatted_inp.get((i,k), None))
        and (i,k) not in found_part_number_indices):
          cur_number_chars.append(formatted_inp[(i,k)])
          cur_number_char_indices.append((i,k))
          if(check_adjacent_symbols(i, k)):
            adjacent_symbol = True
          k += 1
        if adjacent_symbol:
          found_part_numbers.append(int(''.join(cur_number_chars)))
          for pni in cur_number_char_indices:
            found_part_number_indices.append(pni)
  return found_part_numbers


In [4]:
def get_part_ratios(formatted_inp):

  def is_gear(x):
    return x == '*'

  max_i =  max([x[0] for x in formatted_inp.keys()])
  max_j = max([x[1] for x in formatted_inp.keys()])

  def check_adjacent_symbols(i, j):
    for ii in [i-1,i,i+1]:
      for jj in [j-1, j, j+1]:
        if not (ii == i and jj == j):
          if is_gear(formatted_inp.get((ii,jj), None)):
            return ((ii,jj))
    return False

  gear_to_numbers = {}
  found_part_numbers = []
  found_part_number_indices = []
  for i in range(max_i+1):
    for j in range(max_j+1):
      if is_number(formatted_inp.get((i,j), None)):
        k = j
        cur_number_chars = []
        cur_number_char_indices = []
        adjacent_symbol = False
        adjacent_gear_loc = None
        while (is_number(formatted_inp.get((i,k), None))
        and (i,k) not in found_part_number_indices):
          cur_number_chars.append(formatted_inp[(i,k)])
          cur_number_char_indices.append((i,k))
          if(check_adjacent_symbols(i, k) is not False):
            adjacent_symbol = True
            adjacent_gear_loc = check_adjacent_symbols(i, k)
          k += 1
        if adjacent_symbol:
          found_part_numbers.append(int(''.join(cur_number_chars)))
          if adjacent_gear_loc not in gear_to_numbers.keys():
            gear_to_numbers[adjacent_gear_loc] = []
          gear_to_numbers[adjacent_gear_loc].append(int(''.join(cur_number_chars)))
          for pni in cur_number_char_indices:
            found_part_number_indices.append(pni)
  # Filter out only gears with 2 adjacent numbers
  gear_to_numbers = dict(filter(lambda p: len(p[1])==2, gear_to_numbers.items()))

  gear_ratios = [x[0] * x[1] for x  in gear_to_numbers.values()]
  return gear_ratios

In [5]:
import time

t = time.time()

formatted_inp = format_input(inp)

print(sum(get_part_numbers(formatted_inp)))
print(sum(get_part_ratios(formatted_inp)))

print("\nRUNTIME: ", time.time()-t)

546563
91031374

RUNTIME:  0.35807132720947266
